In [ ]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import confusion_matrix

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Current CUDA device: {torch.cuda.current_device()}")
print(f"Device name: {torch.cuda.get_device_name(0)}")

class PointSampler:
    """Uniformly samples points from mesh surfaces using barycentric coordinates"""
    def __init__(self, output_size):
        self.output_size = output_size

    def triangle_area(self, pt1, pt2, pt3):
        """Calculate triangle area using Heron's formula"""
        a = np.linalg.norm(pt1 - pt2)
        b = np.linalg.norm(pt2 - pt3)
        c = np.linalg.norm(pt3 - pt1)
        s = (a + b + c) / 2
        return max(s * (s - a) * (s - b) * (s - c), 0) ** 0.5

    def sample_point(self, pt1, pt2, pt3):
        """Sample random point using barycentric coordinates"""
        s, t = sorted([random.random(), random.random()])
        return (s * pt1[0] + (t-s)*pt2[0] + (1-t)*pt3[0],
                s * pt1[1] + (t-s)*pt2[1] + (1-t)*pt3[1],
                s * pt1[2] + (t-s)*pt2[2] + (1-t)*pt3[2])

    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.array([self.triangle_area(verts[f[0]], verts[f[1]], verts[f[2]]) 
                         for f in faces])
        
        # Sample faces according to their area
        sampled_faces = random.choices(
            faces, weights=areas, k=self.output_size
        )
        
        # Sample points from selected faces
        return np.array([self.sample_point(verts[f[0]], verts[f[1]], verts[f[2]]) 
                        for f in sampled_faces])

class Normalize:
    """Normalize point cloud to zero mean and unit sphere"""
    def __call__(self, pointcloud):
        centroid = np.mean(pointcloud, axis=0)
        pointcloud -= centroid
        max_dist = np.max(np.linalg.norm(pointcloud, axis=1))
        return pointcloud / max_dist

class ToTensor:
    """Convert numpy array to PyTorch tensor"""
    def __call__(self, pointcloud):
        return torch.from_numpy(pointcloud).float()

def default_transforms():
    return transforms.Compose([
        PointSampler(1024),
        Normalize(),
        ToTensor()
    ])

def read_off(file_path):
    """Parse OFF file to extract vertices and faces"""
    with open(file_path, 'r') as f:
        if 'OFF' != f.readline().strip():
            raise ValueError('Not a valid OFF header')
        n_verts, n_faces, _ = map(int, f.readline().strip().split())
        verts = [[float(s) for s in f.readline().strip().split()] 
                for _ in range(n_verts)]
        faces = [[int(s) for s in f.readline().strip().split()][1:] 
                for _ in range(n_faces)]
    return verts, faces

def preprocess_and_save(data_path):
    """Preprocess and save point clouds to .pt files"""
    class_folders = [f for f in os.listdir(data_path) 
                    if os.path.isdir(os.path.join(data_path, f))]
    
    for class_name in tqdm(class_folders, desc="Processing classes"):
        for split in ['train', 'test']:
            src_dir = os.path.join(data_path, class_name, split)
            if not os.path.exists(src_dir):
                continue
                
            for file_name in tqdm(os.listdir(src_dir), desc=f"{class_name}/{split}"):
                if not file_name.endswith('.off'):
                    continue
                    
                # Check if processed file exists
                base_name = file_name[:-4]
                dest_path = os.path.join(src_dir, f"{base_name}.pt")
                if os.path.exists(dest_path):
                    continue
                
                # Process and save
                verts, faces = read_off(os.path.join(src_dir, file_name))
                pointcloud = PointSampler(1024)((verts, faces))
                pointcloud = Normalize()(pointcloud)
                torch.save(pointcloud, dest_path)

class ModelNet40(Dataset):
    """ModelNet40 dataset loader"""
    def __init__(self, root_dir, split='train', transform=None):
        self.root = root_dir
        self.classes = [d for d in os.listdir(root_dir) 
                       if os.path.isdir(os.path.join(root_dir, d))]
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        self.samples = []
        
        # Collect samples
        for cls in self.classes:
            cls_dir = os.path.join(root_dir, cls, split)
            if not os.path.exists(cls_dir):
                continue
                
            for fname in os.listdir(cls_dir):
                if fname.endswith('.pt'):
                    self.samples.append((
                        os.path.join(cls_dir, fname),
                        self.class_to_idx[cls]
                    ))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        return {'pointcloud': torch.load(path), 'label': label}

class PointNet(nn.Module):
    """PointNet architecture for classification"""
    def __init__(self, num_classes=40):
        super().__init__()
        
        # Shared MLP layers
        self.mlp = nn.Sequential(
            nn.Conv1d(3, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 128, 1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 1024, 1),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
        )
        
        # Classification head
        self.fc = nn.Sequential(
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        
    def forward(self, x):
        x = x.transpose(1, 2)  # (B, 3, N)
        x = self.mlp(x)
        x = torch.max(x, 2)[0]  # Global max pooling
        return self.fc(x)

def train_epoch(model, loader, optimizer, criterion):
    """Single training epoch"""
    model.train()
    total_loss = 0.0
    correct = 0
    
    for batch in tqdm(loader, desc="Training", leave=False):
        points = batch['pointcloud'].to(device)
        labels = batch['label'].to(device)
        
        optimizer.zero_grad()
        outputs = model(points)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
    
    return total_loss/len(loader), correct/len(loader.dataset)

def validate(model, loader, criterion):
    """Validation epoch"""
    model.eval()
    total_loss = 0.0
    correct = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc="Validating", leave=False):
            points = batch['pointcloud'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(points)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return total_loss/len(loader), correct/len(loader.dataset), all_preds, all_labels

def plot_confusion_matrix(cm, classes, normalize=True, title='Confusion Matrix'):
    """Plot normalized confusion matrix"""
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=(20, 20))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), 
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

def main():
    # Configuration
    DATA_PATH = '/kaggle/input/modelnet40-princeton-3d-object-dataset/ModelNet40'
    BATCH_SIZE = 64
    EPOCHS = 100
    LR = 0.001
    
    # Preprocess data
    preprocess_and_save(DATA_PATH)

    # Create datasets
    train_set = ModelNet40(DATA_PATH, 'train')
    test_set = ModelNet40(DATA_PATH, 'test')

    # Create data loaders
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, 
                             shuffle=True, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_set, batch_size=BATCH_SIZE,
                            num_workers=4, pin_memory=True)

    # Initialize model
    model = PointNet().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=5)
    criterion = nn.CrossEntropyLoss()

    # Training loop
    best_acc = 0.0
    for epoch in range(EPOCHS):
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc, _, _ = validate(model, test_loader, criterion)
        scheduler.step(val_acc)
        
        print(f"Epoch {epoch+1}/{EPOCHS}:")
        print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc*100:.2f}%")
        print(f"Val Loss: {val_loss:.4f} | Acc: {val_acc*100:.2f}%")
        
        # Save best model
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pth')
            print("Saved new best model!")
            
    # Final evaluation
    model.load_state_dict(torch.load('best_model.pth'))
    _, test_acc, preds, labels = validate(model, test_loader, criterion)
    print(f"\nFinal Test Accuracy: {test_acc*100:.2f}%")

    # Confusion matrix
    cm = confusion_matrix(labels, preds)
    plot_confusion_matrix(cm, train_set.classes)
    plt.show()

if __name__ == "__main__":
    main()